In [1]:
import pandas as pd
import requests
import openai
import os
import sys
import plotly.express as px
import json
from typing import List
from pydantic import BaseModel, Field
sys.path.append("../")
from dotenv import load_dotenv
from finance_functions import *

In [2]:
# Load the environment variables from .env file
load_dotenv()
openai.api_key = os.environ['OPENAI_API_KEY']

In [3]:
from langchain.prompts import ChatPromptTemplate
from langchain_openai import ChatOpenAI
from langchain.schema.output_parser import StrOutputParser

In [4]:
import langchain
print(langchain.__version__)

0.3.24


In [5]:
from langchain_core.tools import tool

In [6]:
get_ticker.invoke("Unicredit")

'UCG.MI'

In [7]:
from langchain_core.messages import HumanMessage

In [17]:
query = "What are the financial of Apple?"

messages = [HumanMessage(query)]

In [18]:
from langchain.chat_models import init_chat_model

llm = init_chat_model("gpt-4.1-nano-2025-04-14", model_provider="openai", temperature=0)

In [19]:
tools = [get_ticker, STOCK, get_stock_prices, get_financials, get_multiple_financials, get_institutional_holders,]

In [20]:
llm_with_tools = llm.bind_tools(tools)

In [21]:
ai_msg = llm_with_tools.invoke(messages)

In [22]:
print(ai_msg.tool_calls)

messages.append(ai_msg)

[{'name': 'get_financials', 'args': {'ticker': 'AAPL'}, 'id': 'call_ZTzkBrC0Hhr1Bj6oe3FRayGj', 'type': 'tool_call'}]


In [23]:
messages

[HumanMessage(content='What are the financial of Apple?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZTzkBrC0Hhr1Bj6oe3FRayGj', 'function': {'arguments': '{"ticker":"AAPL"}', 'name': 'get_financials'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 564, 'total_tokens': 582, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQZgFMyn51JOovXAHvGiXzCd1QY8N', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-100465af-028b-4fd2-9174-f5ce5abf1f97-0', tool_calls=[{'name': 'get_financials', 'args': {'ticker': 'AAPL'}, 'id': 'call_ZTzkBrC0Hhr1Bj6oe3FRayGj', 'type': 'tool_call'}], usage_metadata={'inpu

In [28]:
for tool_call in ai_msg.tool_calls:
    selected_tool = {"get_ticker": get_ticker, "get_financials": get_financials, "STOCK": STOCK, 
                     "get_institutional_holders": get_institutional_holders, "get_multiple_financials": get_multiple_financials,
                     "get_stock_prices": get_stock_prices,
                     }[tool_call["name"].lower()]
    tool_msg = selected_tool.invoke(tool_call)
    messages.append(tool_msg)

messages

[HumanMessage(content='What are the financial of Apple?', additional_kwargs={}, response_metadata={}),
 AIMessage(content='', additional_kwargs={'tool_calls': [{'id': 'call_ZTzkBrC0Hhr1Bj6oe3FRayGj', 'function': {'arguments': '{"ticker":"AAPL"}', 'name': 'get_financials'}, 'type': 'function'}], 'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 18, 'prompt_tokens': 564, 'total_tokens': 582, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 0, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45', 'id': 'chatcmpl-BQZgFMyn51JOovXAHvGiXzCd1QY8N', 'finish_reason': 'tool_calls', 'logprobs': None}, id='run-100465af-028b-4fd2-9174-f5ce5abf1f97-0', tool_calls=[{'name': 'get_financials', 'args': {'ticker': 'AAPL'}, 'id': 'call_ZTzkBrC0Hhr1Bj6oe3FRayGj', 'type': 'tool_call'}], usage_metadata={'inpu

In [29]:
tool_msg

ToolMessage(content='                                                        2024-09-30  \\\nTaxEffectOfUnusualItems                                        0.0   \nTaxRateForCalcs                                              0.241   \nNormalizedEBITDA                                    134661000000.0   \nNetIncomeFromContinuingOperationNetMinorityInte...   93736000000.0   \nReconciledDepreciation                               11445000000.0   \nReconciledCostOfRevenue                             210352000000.0   \nEBITDA                                              134661000000.0   \nEBIT                                                123216000000.0   \nNetInterestIncome                                              NaN   \nInterestExpense                                                NaN   \nInterestIncome                                                 NaN   \nNormalizedIncome                                     93736000000.0   \nNetIncomeFromContinuingAndDiscontinuedOperation      93